# Assignment 3

### Solving the MAXSAT problem with local search algorithms



Let's start with a few definitions:  

Our focus in this assignment is on **Boolean formulas**.
A Boolean formula is constructed
using **variables** that take on the values `True` or
`False` and the operators AND (conjunction, denoted by
$\wedge$), OR (disjunction, denoted by $\vee$), and negation (denoted
by $\neg$).  

We need a few more definitions:
* A **literal** is either a variable (a positive literal), or the
negation of a variable (a negative literal). 
* A **clause** is a disjunction of literals or a single literal.
* A formula is in **conjunctive normal form** (CNF) if it is a conjunction of clauses or a single clause.

For example, the formula $(x_1 \vee x_3) \wedge (x_2 \vee \neg x_3
\vee x_4) \wedge (\neg x_1 \vee x_2 \vee \neg x_4)$ is in CNF.
The formulas we will use in this assignment all have 3 literals per
clause.

The **satisfiability problem (SAT)** is the problem of finding whether there is
an assignment of truth values to the variables which satisfies a
formula in CNF, i.e. makes the formula evaluate to True.
This problem is known to be NP-complete.
We'll consider a more general variant of the problem, which is [MAXSAT](https://en.wikipedia.org/wiki/Maximum_satisfiability_problem): given a formula in CNF, find an
assignment that maximizes the number of satisfied clauses (a clause is
satisfied if there is an assignment of truth values for which it
evaluates to True).
This problem is also NP-complete.


We will encode the instance of `MAXSAT` in a file format 
described below (see also the [2016 SAT competition webpage](http://maxsat.ia.udl.cat/requirements/)).
Here is a ([link](https://www.cs.colostate.edu/~cs440/fall19/assignments/maxsat.tgz)) to a set of 20 instances of MAXSAT that you are asked to use in this assignment.

The MAXSAT instances have a format demonstrated by the following example:

```
c
c start with comments
c
c 
p cnf 5 3
1 -5 4 0
-1 5 3 0
-3 -4 2 0
```

* The file will start with comments, that is lines begining with the character c.
* Right after the comments, there is the line `p cnf nbvar nbclauses` indicating that the instance is in CNF format; `nbvar` is the  number of variables appearing in the file; `nbclauses` is the  number of clauses contained in the file.
* Then the clauses follow. Each clause is a sequence of distinct non-null numbers between `-nbvar` and `nbvar` ending with a 0 (the 0 does not refer to any variable); it cannot contain the opposite literals i and -i simultaneously. A positive number denotes the corresponding variable. A negative number denotes the negation of the corresponding variable.

With that in mind, the above instance of MAXSAT describes the following formula:

$(x_1 \vee \neg x_5 \vee x_4) \wedge (\neg x_1 \vee x_5
\vee x_3) \wedge (\neg x_3 \vee \neg x_4 \vee x_2)$ 

For this formula, there is an assignment of truth values that satisfies all the clauses, so the MAXSAT value for this instance of the problem is 3.

Your task is to solve the MAXSAT problem with local search strategies.
To do that, write a class called `MAXSAT` which is a sub-class
of `search.Problem`.  
The constructor of your `MAXSAT` class should receive a
file name as a parameter and have other methods needed for hill climbing and other methods to work.

```python
import search
class MAXSAT (search.Problem):
    def __init__(self, file_name):
        """Construct an instance of MAXSAT by parsing an instance
        of MAXSAT represented in the given file
        """
        pass
    ## more methods here.
```

## Evaluating solution strategies for MAXSAT

Your task is to compare the following solution strategies for the MAXSAT problem:

* Hill climbing.
* Hill climbing with random restarts (use 20 restarts).
* Genetic algorithms.
* Simulated annealing.

Use the same method for choosing the initial state and neighborhood of
a state for all strategies (for GAs you will need crossover and
mutation operators in addition).
To compare the search strategies run each of these solvers on the provided instances of MAXSAT and compute the average value of the solutions and their running time.
Describe in detail your choice of initial state and neighborhood and
the details of your GA crossover and mutation operators.
Compute the average quality of the solution and the running time for each strategy, treating instances of 50 variables and 150 variables separately.
Explain the difference you observe in performance (how good is the state returned by the algorithm, and how long did
each strategy take to run).
From your experiments, can you tell if the MAXSAT problem has local maxima?
Results should be provided in easy to read tables.

In designing your code, consult the examples shown in class on how to solve problems such as Eight Queens and TSP.  You may use the implementations provided in `search.py`.

For timing python code you can go as simple as:

```Python
import time
t0 = time.time()
code_block
total = time.time() - t0
```

Your solution should appear in the following cells:

*Describe your approach to formulating the MAXSAT problem, how each of the methods will be used to address it.  Your description should include the specification of the cost function, state, neighborhood, and mutation/crossover operators for the GA*.

In [1]:
import search
import os
import time
import random

class MAXSAT(search.Problem) :
    def __init__(self, file_name, random_initial=False) :
        '''The constructor for the MAXSAT problem. This is a satisfiablility problem that
        evaluates variables in conjunctive normal form and tries to find a solution so that
        every clause evaluates to true. In a genetic algorithm problem, the "state" functions
        as the "population". This function reads a file's contents to get the number of variables and
        clauses, and sets an initial array of boolean values. The array of boolean values will
        either be set to all false if there is no random_initial parameter passed in, or all
        random boolean values if it is. In this class "state" or "initial" contains the boolean
        values for [X_1, X_2, X_3, ...] in an array of integers as 0 or 1. The X_i's represent
        variables that are used to evaluate the clauses.'''
        self.clauses = []
        f = open(file_name, 'r')
        contents = f.readlines()
        for line in contents :
            if line[0] == 'c' :
                continue # Do nothing, it's a comment
            elif line[0] == 'p' :
                data = line.split()
                self.num_vars = int(data[2])
                self.num_clauses = int(data[3])
            else :
                data = line.split()
                self.clauses.append(tuple(int(i) for i in data[:len(data)-1]))
        f.close()
        if not random_initial :
            self.initial = [0]*self.num_vars
        else :
            self.initial = [int(random.getrandbits(1)) for i in range(self.num_vars)]
    
    def actions(self, state) :
        '''Returns a tuple containing 1,2,3... to the length of the state. It represents
        all of the X_i indexes you can flip in the state to try to find which is the most
        valuable state. This also acts as the neighborhood fuction in a genetic algorithm
        problem that returns the population.'''
        return tuple(n for n in range(len(state)))

    def result(self, state, action) :
        '''The result function simply flips the value of X_i (given to us by the action)
        in the state and returns it.'''
        result = list(state)
        result[action] = int(not state[action])
        return tuple(result)

    def path_cost(self, c, state1, action, state2) :
        '''Returns the path cost from one state to another given a certain action.'''
        return c + 1 #+ (len(self.clauses) - self.value(state1) - self.value(state2))

    def value(self, state) :
        '''Returns how many clauses evaluate to true in the satisfiability problem with the
        current state. For a genetic algorithm this acts as the fitness function, which
        returns an individual's score.'''
        value = 0
        for clause in self.clauses :
            for v in clause :
                if v < 0 :
                    if not state[abs(v)-1] :
                        value += 1
                        break
                elif state[v-1] :
                    value += 1
                    break
        return value

In [2]:
def run_strategy(strategy_name, files, strategy, num_restarts=0) :
    print('\033[1m' + strategy_name + '\033[0m:\n')
    total_time = 0
    hcs = 0
    nodes_visited = 0
    files_tried = 0
    for f in files :
        random_initial = False
        for i in range(num_restarts + 1) :
            files_tried += 1
            maxsat = MAXSAT(f, random_initial)
            inst = search.InstrumentedProblem(maxsat)
            # Get time it takes to complete the search
            start = time.time()
            result = strategy(inst)
            total_time += (time.time() - start)
            # Count the total number of nodes visited
            nodes_visited += inst.succs
            # Count if the goal was found
            if inst.found :
                hcs = maxsat.value(result)
                break
            value = maxsat.value(result)
            if value > hcs :
                hcs = value
            random_initial = True
    print('# of files:', len(files))
    print('Total time:', total_time, 'sec')
    print('Average time:', total_time/files_tried, 'sec')
    print('Average nodes visited:', nodes_visited/files_tried)
    if num_restarts > 0 :
        print('# of restarts:', num_restarts)
    print('Highest clause satisfaction:', hcs)
    print()

def hill_climbing(problem):
    current = search.Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = search.argmax(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
    return current.state

def iterative_hill_climbing(problem):
    current = search.Node(problem.initial)
    i = 0
    while i < 30:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = search.argmax(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
        i += 1
    return current.state

def run_genetic_strategy(strategy_name, files) :
    print('\033[1m' + strategy_name + '\033[0m:\n')
    total_time = 0
    hcs = 0
    files_tried = 0
    for f in files :
        maxsat = MAXSAT(f, True)
        num_population = 25
        population = search.init_population(num_population, [0,1], maxsat.num_vars)
        # Get time it takes to complete the search
        start = time.time()
        result = search.genetic_algorithm(population, maxsat.value, f_thres=200)
        total_time += (time.time() - start)
        # Keep track of the highest clause satisfaction
        value = maxsat.value(result)
        if value > hcs :
            hcs = value
        random_initial = True
    print('# of files:', len(files))
    print('Total time:', total_time, 'sec')
    print('Average time:', total_time/len(files), 'sec')
    print('Highest clause satisfaction:', hcs)
    print()

# File initialization
filenames = [d for d in os.listdir('.') if d[0] != '.']
filenames.sort()
filenames50 = [f for f in filenames if f[:5] == 'uuf50']
filenames150 = [f for f in filenames if f[:6] == 'uuf150']

# Hill climbing
run_strategy('Hill climbing (50)', filenames50, hill_climbing)
run_strategy('Hill climbing (150)', filenames150, hill_climbing)

# Hill climbing with random restarts
run_strategy('Hill climbing, random restarts (50)', filenames50, iterative_hill_climbing, 20)
run_strategy('Hill climbing, random restarts (150)', filenames150, iterative_hill_climbing, 20)

# Genetic
run_genetic_strategy('Genetic algorithm (50)', filenames50)
run_genetic_strategy('Genetic algorithm (150)', filenames150)

# Simulated annealing
run_strategy('Simulated annealing (50)', filenames50, search.simulated_annealing)
run_strategy('Simulated annealing (150)', filenames150, search.simulated_annealing)

Hill climbing (50):

# of files: 10
Total time: 0.2906193733215332 sec
Average time: 0.02906193733215332 sec
Average nodes visited: 12.4
Highest clause satisfaction: 215

Hill climbing (150):

# of files: 10
Total time: 5.9587178230285645 sec
Average time: 0.5958717823028564 sec
Average nodes visited: 33.6
Highest clause satisfaction: 633

Genetic algorithm (50):

# of files: 10
Total time: 0.4096558094024658 sec
Average time: 0.04096558094024658 sec
Highest clause satisfaction: 204

Genetic algorithm (150):

# of files: 10
Total time: 0.7660274505615234 sec
Average time: 0.07660274505615235 sec
Highest clause satisfaction: 591

Simulated annealing (50):

# of files: 10
Total time: 0.19146943092346191 sec
Average time: 0.019146943092346193 sec
Average nodes visited: 100.0
Highest clause satisfaction: 196

Simulated annealing (150):

# of files: 10
Total time: 0.6285920143127441 sec
Average time: 0.06285920143127441 sec
Average nodes visited: 100.0
Highest clause satisfaction: 582



## Results (50):
|                             | Hill climbing | Hill climbing, random restarts | Genetic | Simulated annealing |
|-|-|-|-|-|
| # of files                  | 10            | 10                             | 10      | 10                  |
| Total time (sec)            | 0.4384        | 6.9465                         | 1.9444  | 0.3296              |
| Average time (sec)          | 0.0438        | 0.0331                         | 0.1944  | 0.0330              |
| Average nodes visited       | 12.4          | 11.8811                        | N/A     | 100                 |
| Highest clause satisfaction | 215           | 217                            | 202     | 201                 |

## Results (150):
|                             | Hill climbing | Hill climbing, random restarts | Genetic | Simulated annealing |
|-|-|-|-|-|
| # of files                  | 10            | 10                             | 10      | 10                  |
| Total time (sec)            | 8.6005        | 152.7677                       | 1.2958  | 1.1430              |
| Average time (sec)          | 0.8600        | 0.7275                         | 0.1296  | 0.1143              |
| Average nodes visited       | 33.6          | 29.6333                        | N/A     | 100                 |
| Highest clause satisfaction | 633           | 636                            | 586     | 582                 |

## Discussion:
As you can see, both versions of hill climbing have better results than the other algorithms, and were close to solving for the solution. Hill climbing did take longer, but if you increase the number of restarts hill climbing would eventually be able to find a solution. I believe hill climbing without restarts was so close to finding a solution because using all false values as an initial state was already pretty close to the solution. Genetic and simulated annealing algorithms had similar total times and highest clause satisfaction (HCS), which was interesting. I suspect it's because I'm using the default scheduling algorithm for simulated annealing. I also found 200 to be a good fitness threshold for the genetic algorithm because it increases the HCS a good amount. But anything above 200 significantly increasing the total time. I used 30 as the number of iterations for hill climbing with random restarts for the same reason, any more increases total time too much, any less reduces HCS. I chose to use the crossover and mutation functions provided in search.py because there's not anything I'd want to change. 

## Submission

Answer the questions in the cells reserved for that purpose.
Submit your report as a Jupyter notebook via Canvas. Running the notebook should generate all the results in your notebook.  Leave the output of your notebook intact so we don't have to run it to see the results.

## Grading

Grading Rubric:

```
25 pts:  Formulation of the MAXSAT problem
50 pts:  Correctness of your code for solving the MAXSAT problem
25 pts:  Discussion of the results
```

Grading will be based on the following criteria:
* Correct behavior of the required code
* Overall readability and organization of the notebook
* Effort in making interesting observations where requested.
* Conciseness. Points may be taken off if the notebook is overly long.